In [1]:
import findspark

In [2]:
findspark.init('/home/dhruva_stark/spark-3.4.1-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark=SparkSession.builder.appName('nlp').getOrCreate()

23/07/12 18:37:59 WARN Utils: Your hostname, linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/12 18:37:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/12 18:38:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/12 18:38:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/12 18:38:02 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
data=spark.read.csv('smsspamcollection/SMSSpamCollection',inferSchema=True,sep='\t')

In [7]:
data=data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [8]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import length

In [10]:
data=data.withColumn('length',length(data['text']))

In [11]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [12]:
data.groupBy('class').mean().show()


+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [13]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer)

In [14]:
tokenizer=Tokenizer(inputCol='text',outputCol='token_text')
stop_remove=StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec=CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf=IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric=StringIndexer(inputCol='class',outputCol='label')

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
clean_up=VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [17]:
from pyspark.ml import Pipeline

In [18]:
data_prep_pipe=Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [19]:
cleaner=data_prep_pipe.fit(data)

In [20]:
clean_data=cleaner.transform(data)

In [21]:
clean_data=clean_data.select('label','features')

In [22]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [23]:
from pyspark.ml.classification import LogisticRegression


In [24]:
lr=LogisticRegression()

In [25]:
training,test=clean_data.randomSplit([0.7,0.3])


In [26]:
spam_detection=lr.fit(training)

23/07/12 18:43:09 WARN DAGScheduler: Broadcasting large task binary with size 1165.9 KiB
23/07/12 18:43:10 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/12 18:43:10 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
23/07/12 18:43:12 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
23/07/12 18:43:13 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
23/07/12 18:43:13 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
23/07/12 18:43:14 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
23/07/12 18:43:14 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
23/07/12 18:43:15 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
23/07/12 18:43:15 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
23/07/12 18:43:16 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
23/

In [27]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [28]:
test_results=spam_detection.transform(test)

In [29]:
test_results.show()

23/07/12 18:43:40 WARN DAGScheduler: Broadcasting large task binary with size 1271.6 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[22.7050184058899...|[0.99999999986217...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[15.3258952674627...|[0.99999977917504...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[24.6190925181831...|[0.99999999997967...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[10.0507320912837...|[0.99995684771748...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[20.6453111929993...|[0.99999999891892...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[24.1743075457983...|[0.99999999996828...|       0.0|
|  0.0|(13424,[0,1,14,18...|[23.1445105133842...|[0.99999999991118...|       0.0|
|  0.0|(13424,[0,1,20,27...|[14.9323119326373...|[0.99999967267499...|       0.0|
|  0.0|(13424,[0,1,21,27...|[15.5435255778177...|[0.99999982236336...|       0.0|
|  0.0|(13424,[0

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
acc_eval=MulticlassClassificationEvaluator()

In [32]:
acc = acc_eval.evaluate(test_results)



23/07/12 18:44:02 WARN DAGScheduler: Broadcasting large task binary with size 1276.2 KiB


In [34]:
print('ACC of lr Model')
print(acc)


ACC of lr Model
0.9862372370719219
